# 基于寒武纪 MLU 的模型推理--Tacotron2 语音合成
### -- PyTorch，CNNL 推理，Python，FP32
## 目录
### 0 基本信息
### 1 实验内容及目标
     1.1 实验内容
     1.2 实验目标
### 2 前置知识介绍
     2.1 寒武纪软硬件平台介绍
     2.2 寒武纪 PyTorch 介绍
     2.3 CNNL 介绍
### 3 理解原始模型
     3.1 模型概述
     3.2 Tacotron2 网络梳理
### 4 编写推理应用
     4.1 工程目录介绍
     4.2 推理步骤简析
     4.3 工程准备      
        4.3.1 原始环境准备
        4.3.2 代码修改
        4.3.3 确定待合成的文本
        4.3.4 安装环境依赖项
     4.4 模型推理
        4.4.1 设定超参
        4.4.2 文本前处理
        4.4.3 加载模型
        4.4.4 文本特征序列生成 mel 谱
        4.4.5 基于 WaveGlow 从 mel 谱生成 audio
        4.4.6 移除 WaveGlow 的 bias（可选）
        4.4.7 一键运行推理

### 5 结语
     5.1 回顾重点步骤
     5.2 相关链接

# 0 基本信息

发布者：寒武纪

实验时长：90分钟

语言：Python3

修改时间：2022-08-15

# 1 实验内容及目标
## 1.1 实验内容

1. 本实验主要介绍基于寒武纪 MLU370 硬件平台及寒武纪 PyTorch 框架的 Tacotron2 语音合成推理应用的开发方法。

2. 基于 Tacotron2 语音合成应用和寒武纪 MLU370 硬件平台，以文本作为输入，合成类人的语音输出。

## 1.2 实验目标

1. 掌握使用寒武纪 MLU370 硬件平台及寒武纪 PyTorch 框架进行 AI 模型移植与推理的基本方法。

2. 理解 Tacotron2 模型和 WaveGlow 模型的网络结构。


# 2 前置知识介绍

## 2.1 寒武纪软硬件平台介绍

- 硬件：寒武纪 MLU370 AI 计算卡   
 
- AI 框架：寒武纪 PyTorch 1.6  
 
- 寒武纪计算库：CNNL 1.10.5  
 
- 系统环境：寒武纪云平台 

## 2.2 寒武纪 PyTorch 介绍

为支持寒武纪 MLU，寒武纪定制了开源 AI 编程框架 PyTorch (以下简称 Cambricon PyTorch)。Cambricon PyTorch 借助 PyTorch 自身提供的设备扩展接口将 MLU 后端库中所包含的算子操作动态注册到 PyTorch 中，使 AI 模型可运行在 MLU 上。
    Cambricon PyTorch 已基于 CNNL 库在 MLU 后端实现了大量常用的 AI Network 算子，默认情况下，Cambricon PyTorch 会以 CNNL 的方式运行 AI 模型的各类算子，用户无需关心 CNNL 算子细节与调用方式，只需将模型与数据拷贝到 MLU 设备端即可在 MLU 上运行。

## 2.3 CNNL 介绍

CNNL 是寒武纪自研的高性能 AI 运算库，针对常见 AI 算子提供了高效实现，同时适用训练与推理场景，通常作为 AI 框架的后端实现，用户不直接感知。CNNL 具有灵活、简便、易用等特点。寒武纪 PyTorch 的后端实现默认为 CNNL。


# 3. 理解原始模型

## 3.1 模型概述
Tacotron2 是一个可直接从文本合成语音的端到端语音合成 AI Network 架构，合成的语音可与真人语音媲美。

## 3.2 Tacotron2 网络梳理

Tacotron2 系统由两部分构成，第一部分是循环 seq2seq 结构的特征预测网络，负责把字符向量映射为梅尔声谱图，第一部分后再接一个 WaveNet 模型的修订版，即 WaveGlow，负责把梅尔声谱图合成为时域波形。编码器（下图中的蓝色块）将整个文本转换为固定大小的隐藏特征表示。然后，自回归解码器（橙色方块）使用该特征表示，每次生成一个频谱图帧。在 Tacotron2 中，使用基于流生成的 WaveGlow 代替了自回归 WaveNet（绿色方块）。
从[Tacotron2论文](https://arxiv.org/abs/1712.05884)可知，Tacotron2 网络的结构如下图所示：

 <img src="course_images/tacotron2_structure.bmp" width="40%">

从[WaveGlow 论文](https://arxiv.org/abs/1712.05884)可知，WaveGlow 网络的结构如下图所示。WaveGlow 模型是一种基于流的生成模型，
它从以梅尔谱图为限制条件的高斯分布中生成音频样本。在训练期间，模型学习如何通过一系列 Flow 将数据集分布变换为球形高斯分布。前述 Flow 中的一个步骤包括可逆卷积，然后是作为仿射耦合层的修改过的 WaveNet。在推理过程中，网络被反转，音频样本从高斯分布中生成。

 <img src="course_images/waveglow_net_structure.bmp" width="40%">


# 4 编写推理应用

## 4.1 工程目录介绍

|-- Tacotron2   \
|&emsp;   |--course_images          &emsp;&emsp;                  （存放jupyter所需的图片） \
|&emsp;   |-- requirements.txt       &emsp;&emsp;                 （环境依赖）\
|&emsp;   |-- tacotron2_infer.sh         &emsp;&emsp;                  （推理入口shell脚本） \
|&emsp;   |-- README.md       &emsp;&emsp;                 （使用MLU进行推理的README）\
|&emsp;   |-- src \
|&emsp;&emsp;     |-- common    &emsp;&emsp;        （tacotron2 模块）\
|&emsp;&emsp;&emsp;      |-- stft.py    &emsp;&emsp;        （短时傅里叶变换，应用在denoiser内）\
|&emsp;&emsp;     |-- inference.py   &emsp;&emsp;        （推理代码，提供了模型加载，数据预处理等功能）\
|&emsp;&emsp;     |-- test_infer.py    &emsp;&emsp;        （推理入口python脚本，被tacotron2_infer.sh调用）\
|&emsp;&emsp;     |-- tacotron2    &emsp;&emsp;        （tacotron2 模块）\
|&emsp;&emsp;&emsp;      |-- model.py    &emsp;&emsp;        （tacotron2 模型结构定义）\
|&emsp;&emsp;     |-- waveglow    &emsp;&emsp;        （waveglow 模块）\
|&emsp;&emsp;&emsp;      |-- model.py    &emsp;&emsp;        （waveglow 模型结构定义）\
|&emsp;&emsp;&emsp;      |-- denoiser.py    &emsp;&emsp;        （降噪模块，提升合成的语音质量）\
|&emsp;&emsp;&emsp;      |-- ...... &emsp;&emsp;  
|&emsp;&emsp;     |-- README.md   &emsp;&emsp;        （原始README）\
|&emsp;&emsp;     |--......\
|&emsp;   |--...... \

## 4.2 推理步骤简析
如下图所示，是本次实验的主要步骤。

<img src="course_images/tacotron2_infer_step.png" width="60%">

## 4.3 工程准备

### 4.3.1 原始环境准备

- 下载[原始工程代码](https://github.com/NVIDIA/DeepLearningExamples/tree/46ff3707e04683e41b79af0f94a74e45f8016786/PyTorch/SpeechSynthesis/Tacotron2#quick-start-guide)。

本次实验我们将使用 FP32 精度进行推理，可使用如下预训练模型：

- 下载[tacotron2 fp32 预训练模型](https://catalog.ngc.nvidia.com/orgs/nvidia/models/tacotron2_pyt_ckpt_fp32)至本地保存。
- 下载[waveglow fp32 预训练模型](https://catalog.ngc.nvidia.com/orgs/nvidia/models/waveglow_ckpt_fp32)至本地保存。

In [ ]:
!bash prepare.sh

### 4.3.2 代码修改

原始工程代码并不能直接在寒武纪 PyTorch 内运行，还需进行如下步骤的修改：
1. 修改`tacotron2_infer.sh`内的`WN_CHANNEL`为512。
2. 根据 Tacotron2 与 WaveGlow 预训练模型的本地保存路径，修改`tacotron2_infer.sh`内`TACOTRON2_CKPT`与`WAVEGLOW_CKPT`的值。通常会将预训练模型保存到`practices/model/pretrained/pytorch_tacotron2_inference`。
3. 进入`src/`目录，在`test_infer.py`，`inference.py`, `models.py`内import torch_mlu 库。
4. 在`test_infer.py`内，将`cpu_run` 改为`device_param`以便新增 MLU 设备，并根据`test_infer.py`内涉及到`cpu_run`的函数，分别在相关函数的定义文件中进行修改。涉及到的文件有：`inference.py`, `models.py`, `waveglow/denoiser.py`。以`test_infer.py`为例，涉及到的主要修改如下方所示：

&emsp;(1) 在`parse_args()`内删除`cpu_run`参数的解析，并新增`device_param`参数解析：

```bash
#parser.add_argument('--cpu-run', action='store_true', help='Run inference on CPU')

parser.add_argument('--device-param', type=str, default='mlu', help='inference device, optional val is gpu/mlu/cpu.')
```

&emsp;(2)修改`load_and_setup_model()`函数，将`cpu_run`替换为`device_param`并将模型拷贝至mlu：

```bash
# 注释load_and_setup_model原有定义，改为使用device_param 判断设备信息
# def load_and_setup_model(model_name, parser, checkpoint, amp_run, cpu_run, forward_is_infer=False):
def load_and_setup_model(model_name, parser, checkpoint, amp_run, device_param, forward_is_infer=False):
    model_parser = models.parse_model_args(model_name, parser, add_help=False)
    model_args, _ = model_parser.parse_known_args()
    model_config = models.get_model_config(model_name, model_args)
    # 还需修改models.get_model()，将cpu_run 改为 device_param
    model = models.get_model(model_name, model_config, device_param, forward_is_infer=forward_is_infer)

    if checkpoint is not None:
        # 根据device_param判断当前设备，若是mlu,则先将模型文件加载到cpu，并随后拷贝至mlu
        if device_param != "gpu":
            # cambricon-note:for mlu or cpu
            state_dict = torch.load(checkpoint, map_location=torch.device('cpu'))['state_dict']
        else:
            state_dict = torch.load(checkpoint)['state_dict']

        if checkpoint_from_distributed(state_dict):
            state_dict = unwrap_distributed(state_dict)

        model.load_state_dict(state_dict)

        if device_param == "mlu":
            #模型拷贝至mlu
            cur_device = torch.device('mlu')
            model = model.to(cur_device)

    if model_name == "WaveGlow":
        model = model.remove_weightnorm(model)

    model.eval()

    if amp_run:
        model, _ = amp.initialize(model, [], opt_level="O3")

    return model

```

&emsp;(3) 修改`Denoiser()的调用`：

```bash
#原始代码：
    if args.cpu_run:
        denoiser = Denoiser(waveglow, args.cpu_run)
    else:
        denoiser = Denoiser(waveglow, args.cpu_run).cuda()
# 需要改为如下代码，即判断当前设备是否是mlu，如果是，则需要将Denoiser()的返回值拷贝至mlu，且需要同步修改Denoiser()的传参：
    if args.device_param == "cpu":
        denoiser = Denoiser(waveglow, args.device_param)
    elif args.device_param == "mlu":
        denoiser = Denoiser(waveglow, args.device_param).mlu()
    else:        
        denoiser = Denoiser(waveglow, args.device_param).cuda()

```

&emsp;(4) 其他函数调用的修改与上述代码修改思路一致，在此不再赘述。

5. 修改`src/common/stft.py`，该文件中的`inverse()`函数内的`magnitude`只考虑了 GPU 情形，未考虑 MLU 情形。当`magnitude`在 MLU 上时，`window_sum`相应地也需拷贝至 MLU 上。新增这部分逻辑即可。
6. 由于本实验仅适用于寒武纪 PyTorch 环境，因此需注释掉`src/`内`test_infer.py`与`inference.py`内`apex`相关的代码。

**4.3.1与4.3.2的大部分改动都已提前实现，您只需下载本仓库[代码](http://gitlab.software.cambricon.com/neuware/software/ae/ecosystem/practices.git)，再参考4.3.2中的第2步完成预训练模型的下载与路径指定。**


### 4.3.3 确定待合成的文本

待合成为语音的文本在`src/test_infer.py`内的`main()`中，可根据自己需求对`main()`中的`texts`进行修改。代码内初始文本为：
```bash
    texts = ["The forms of printed letters should be beautiful, and that their arrangement on the page should be reasonable and a help to the shapeliness of the letters themselves. The forms of printed letters should be beautiful, and that their arrangement on the page should be reasonable and a help to the shapeliness of the letters themselves."]
    
```

实际上，代码内会对`texts`按照`args.input_length`和`args.batch_size`进行截断与复制：

```bash
    texts = [texts[0][:args.input_length]]
    texts = texts*args.batch_size
    #args.input_length 默认值为166
    #args.batch_size 默认为1
    # 因此，经过上述处理后，texts为：
    "The forms of printed letters should be beautiful, and that their arrangement on the page should be reasonable and a help to the shapeliness of the letters themselves."

```


### 4.3.4 安装环境依赖项

默认此时已经入`Cambricon PyTorch`虚拟环境。按照下方代码所示，安装环境依赖。

In [ ]:
import os
os.system('pip install -r requirements.txt')
# 只输出CNNL报错信息
os.system('export CNNL_MIN_LOG_LEVEL=3')

## 4.4 模型推理
### 4.4.1 设定超参
正式开始推理之前，我们需要设定相关参数的值，如预训练模型路径、推理设备、输入文本长度等。
下方代码块负责解析命令行透传而来的参数。

In [ ]:
import sys

cur_path = os.getcwd()
models_path = cur_path + "/src/"
sys.path.append(models_path)

from inference import checkpoint_from_distributed, unwrap_distributed, MeasureTime, prepare_input_sequence,load_and_setup_model
import models
import torch
import argparse
import numpy as np
from scipy.io.wavfile import write

import time
import dllogger as DLLogger
from dllogger import StdOutBackend, JSONStreamBackend, Verbosity
# cambricon-note: no apex in cambricon-pytorch
#from apex import amp
from waveglow.denoiser import Denoiser

# cambricon-note: use cambricon-pytorch
import torch_mlu


def parse_args(parser):

    """
    Parse commandline arguments.
    """
    parser.add_argument('--tacotron2', type=str,
                        help='full path to the Tacotron2 model checkpoint file')
    parser.add_argument('--waveglow', type=str,
                        help='full path to the WaveGlow model checkpoint file')
    parser.add_argument('-s', '--sigma-infer', default=0.6, type=float)
    parser.add_argument('-d', '--denoising-strength', default=0.01, type=float)
    parser.add_argument('-sr', '--sampling-rate', default=22050, type=int,
                        help='Sampling rate')
    parser.add_argument('--amp-run', action='store_true',
                        help='inference with AMP')
    parser.add_argument('--log-file', type=str, default='nvlog.json',
                        help='Filename for logging')
    parser.add_argument('--stft-hop-length', type=int, default=256,
                        help='STFT hop length for estimating audio length from mel size')
    parser.add_argument('--num-iters', type=int, default=10,
                        help='Number of iterations')
    parser.add_argument('-il', '--input-length', type=int, default=64,
                        help='Input length')
    parser.add_argument('-bs', '--batch-size', type=int, default=1,
                        help='Batch size')
    parser.add_argument('--device-param', type=str, default='mlu',
                        help='inference device, optional val is gpu/mlu/cpu.')
    return parser


#------------ main --------------------------------
parser = argparse.ArgumentParser(description='PyTorch Tacotron 2 Inference')
parser = parse_args(parser)
cur_path = os.getcwd()
pretrained_model_dir = cur_path + "/../../model/pretrained/pytorch_tacotron2_inference/"

args, unknown_args = parser.parse_known_args()
# 设定预训练模型路径，推理设备，waveglow_channel, 输入文本的input_length
args.tacotron2 = pretrained_model_dir + "nvidia_tacotron2pyt_fp32_20190427"
args.waveglow = pretrained_model_dir + "nvidia_waveglowpyt_fp32_20190427"
args.device_param = "mlu"
args.input_length=166  # 待合成的文本长度，下面例子中一句话长度是 166
args.batch_size=1
args.num_iters=14
# wn_channels 参数的解析在models.py完成
args.wn_channels=512

print("args:",args,unknown_args)


### 4.4.2 文本前处理

如 4.4.1所述，初始文本会经过`args.input_length`与`args.batch_size`的截断与复制。由4.2可知，在前处理阶段，Tacotron2 会基于`texts`生成数字序列(sequences_padded)。我们看下前处理过程的输入输出：

In [ ]:
texts = ["The forms of printed letters should be beautiful, and that their arrangement on the page should be reasonable and a help to the shapeliness of the letters themselves. The forms of printed letters should be beautiful, and that their arrangement on the page should be reasonable and a help to the shapeliness of the letters themselves."]
texts = [texts[0][:args.input_length]]
texts = texts * args.batch_size
sequences_padded, input_lengths = prepare_input_sequence(texts, args.device_param)
print("texts before pre-process:{}".format(texts))
print("texts after pre-process:{}".format(sequences_padded))

### 4.4.3 加载模型
完成超参设定后，还需根据传入的命令行参数加载模型，并将模型拷贝至 MLU 上。

In [ ]:
tacotron2 = load_and_setup_model('Tacotron2', parser, args.tacotron2, args.amp_run, args.device_param, forward_is_infer=True)
waveglow = load_and_setup_model('WaveGlow', parser, args.waveglow, args.amp_run, args.device_param)

if args.device_param == "cpu":
    denoiser = Denoiser(waveglow, args.device_param)
elif args.device_param == "mlu":
    denoiser = Denoiser(waveglow, args.device_param).mlu()
else:
    denoiser = Denoiser(waveglow, args.device_param).cuda()
    

### 4.4.4 文本特征序列生成 mel 谱

在对输入文本完成前处理得到文本序列特征`sequences_padded`后，即可开始基于`sequences_padded`生成 mel 谱。

In [ ]:

# 生成mel 谱
with torch.no_grad():
    mel, mel_lengths, alignments = tacotron2(sequences_padded, input_lengths)



使用如下代码即可画出 mel 谱和 alignments。

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

# 定义 画图函数，在jupyter notebook内显示
def plot_data(data,figsize=(16,4)):    
    fig,axes = plt.subplots(1,len(data),figsize=figsize)
    for i in range(len(data)):            
        axes[i].imshow(data[i],aspect='auto',origin='lower',interpolation='none')
        
#将mel数据与alignment数据从mlu拷贝回cpu,并取后两维
mel_for_plot = mel.float().data.cpu()[0]
alignments_for_plot = alignments.float().data.cpu()[0]
plot_data((mel_for_plot,alignments_for_plot))


### 4.4.5 基于 WaveGlow 从 mel 谱生成 audio
得到 mel 谱后，便可利用 WaveGlow 生成 audio 数据，并将 audio 数据保存到本地。在本实验中，将直接展示合成的 audio 音频，运行下方代码块后，即可得到支持在线播放的 audio 音频。


In [ ]:
import IPython
import IPython.display as ipd
with torch.no_grad():
    audios = waveglow.infer(mel, sigma=args.sigma_infer)
    audios_float = audios.float()
    audios_npy = audios_float.cpu().numpy()    
for i, audio in enumerate(audios_npy):        
    # i = 0
    tmp_audio = audios_npy[:mel_lengths[i]*args.stft_hop_length]
ipd.Audio(tmp_audio, rate=args.sampling_rate)

### 4.4.6 移除 WaveGlow 的 bias（可选）
移除 WaveGlow 的 bias 后可能可以得到更加清晰的音质。

In [ ]:
audios_denoiser = denoiser(audios_float, strength=args.denoising_strength).squeeze(1)
audios_denoiser_npy = audios_denoiser.cpu().numpy()
audios_npy = audios_float.cpu().numpy()
ipd.Audio(audios_denoiser_npy, rate=args.sampling_rate)

### 4.4.7 一键运行推理
4.4.2至4.4.6只运行了一次推理，合成的语音音质并不够清晰，为了得到更加清晰的音质，您需要进行多次推理。
为方便一键运行，本实验已将4.4.2到4.4.7之间的步骤封装至推理脚本`tacotron2_infer.sh`，在该脚本中设置好预训练模型路径与`batch_size`，推理次数`NUM_ITERS`后即可运行得到音频文件。
`tacotron2_infer.sh`内的推理迭代次数`NUM_ITERS`默认为`23`，其中 warmup 推理`3`次，因此，实际推理次数为`20`.通常来说，迭代推理`20`次即可得到一份可用的音频文件。
在本次实验中，我们使用的`batch_size`默认为1.

In [ ]:
os.system('bash tacotron2_infer.sh')
# 预计需要运行一分钟左右
IPython.display.Audio("audio_0.wav")

推理结束后将在当前目录内生成`audio_0.wav`音频文件。

# 5 结语

## 5.1 回顾重点步骤

至此，FP32 精度下，基于寒武纪 PyTorch 的 Tacotron2 推理应用开发实验已经完毕。让我们回顾一下在使用寒武纪 PyTorch 框架和 CNNL 进行推理时，相比于使用原生 PyTorch，都有哪些主要开发步骤：
1. 导入torch_mlu 库。
2. 将模型与数据拷贝到 MLU 上。
由此可见，利用寒武纪 PyTorch 与 CNNL 进行推理应用开发时，所需修改量很少，开发步骤十分简洁，使用体验与原生 PyTorch 完全一致。


## 5.2 相关链接


1. 对上述代码有疑问请提交 ISSUE:  
https://gitee.com/cambricon/practices/issues   

2. 更多与寒武纪开发相关的有趣内容请移步至寒武纪开发者社区：    
https://developer.cambricon.com/

3. 如果有任何其他问题可到寒武纪开发者论坛提问，会有专人为您解答：  
https://forum.cambricon.com//list-1-1.html
